# SVM 支持向量机 分类

In [38]:
# 加载库
%matplotlib inline
import os
import numpy as np
from scipy import stats
import pandas as pd
from sklearn import svm
import sklearn.model_selection as cross_validation
import matplotlib.pyplot as plt

#pd.set_option('display.max_columns', None)

In [39]:
# 数据 assets_rank 为分类字段

In [40]:
# 加载数据
orgData = pd.read_csv('data/date_data2.csv')
orgData.head()

,income,attractive,assets,edueduclass,Dated,income_rank,attractive_rank,assets_rank
0,3000,9.0,5.145476,1,0,0,0,0
1,3000,14.5,40.643781,4,1,0,0,1
2,3000,6.0,5.145476,1,0,0,0,0
3,3000,1.0,7.067434,1,0,0,0,0
4,3500,14.5,3.728400,2,0,0,0,0


In [41]:
orgData.describe()

,income,attractive,assets,edueduclass,Dated,income_rank,attractive_rank,assets_rank
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,9010.000000,50.500000,96.006300,3.710000,0.500000,1.550000,1.560000,1.510000
std,5832.675288,28.810948,91.082226,1.225116,0.502519,1.140397,1.103896,1.123621
min,3000.000000,1.000000,3.728400,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.000000,28.000000,31.665269,3.000000,0.000000,1.000000,1.000000,0.750000
50%,7500.000000,51.000000,70.746924,4.000000,0.500000,2.000000,2.000000,2.000000
75%,11500.000000,68.875000,131.481061,4.000000,1.000000,3.000000,2.250000,2.250000
max,34000.000000,99.500000,486.311758,6.000000,1.000000,3.000000,3.000000,3.000000


In [42]:
# 提取字段
X = orgData.iloc[:, 0:4]
y = orgData['Dated']

In [43]:
X.head()

,income,attractive,assets,edueduclass
0,3000,9.0,5.145476,1
1,3000,14.5,40.643781,4
2,3000,6.0,5.145476,1
3,3000,1.0,7.067434,1
4,3500,14.5,3.728400,2


In [53]:
# 进行标准化，可以提升高斯核svm的表现
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[-1.03559293e+00, -1.44768127e+00, -1.00259478e+00,
        -2.22317935e+00],
       [-1.03559293e+00, -1.25581990e+00, -6.10892242e-01,
         2.37904801e-01],
       [-1.03559293e+00, -1.55233293e+00, -1.00259478e+00,
        -2.22317935e+00],
       [-1.03559293e+00, -1.72675236e+00, -9.81387123e-01,
        -2.22317935e+00],
       [-9.49437109e-01, -1.25581990e+00, -1.01823137e+00,
        -1.40281797e+00],
       [-9.49437109e-01, -7.84887435e-01, -9.20679025e-01,
        -5.82456582e-01],
       [-9.49437109e-01,  5.23258290e-01, -6.10892242e-01,
         2.37904801e-01],
       [-9.49437109e-01, -1.44768127e+00, -9.43202793e-01,
        -1.40281797e+00],
       [-9.49437109e-01, -1.02907464e+00, -6.99584712e-01,
        -1.40281797e+00],
       [-9.49437109e-01, -1.44768127e+00, -9.54262111e-01,
        -2.22317935e+00],
       [-9.49437109e-01, -1.25581990e+00, -8.74089355e-01,
        -5.82456582e-01],
       [-9.49437109e-01, -1.55233293e+00, -8.74089355e-01,
      

In [57]:
# 划分训练集和测试集
train_data, test_data, train_target, test_target = cross_validation.train_test_split(
    X_scaled, y, train_size=0.8, random_state=123)   #划分训练集和测试集

In [60]:
train_data[0:4]

array([[-0.60481384,  1.58721681, -0.61089224,  0.2379048 ],
       [ 1.20445833,  0.52325829,  1.15793837,  1.05826618],
       [-0.77712547, -1.02907464, -0.98138712, -2.22317935],
       [-0.69096966, -0.38372275, -0.02636063,  1.87862757]])

In [61]:
svcModel1 = svm.SVC(kernel='rbf', gamma=0.5, C=0.5, probability=True).fit(train_data, train_target)
# test_est1 = svcModel1.predict(test_scaled)
# print(metrics.classification_report(test_target, test_est1))  # 计算评估指标


##  模型选优

In [66]:
#选择最优模型
#https://blog.csdn.net/u014652309/article/details/84979497

from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn import metrics
# 使用两个核函数因为不知道哪个好。要每种组合都试一遍，选择最优的，（f1-score）最大的。
kernel = ['linear', 'rbf']
gamma =  [0.5]#np.arange(0.01, 1, 0.1)
print(gamma)
C =[0.5] #np.arange(0.01, 1.0, 0.1)
print(C)
grid = {'gamma': gamma, 'C': C,'kernel': kernel}
clf_search = GridSearchCV(estimator=svcModel1, scoring='precision',param_grid=grid, cv=4)
clf_search.fit(train_data, train_target)
# GridSearchCV?
clf_search.best_estimator_

best_model = clf_search.best_estimator_
test_est2 = best_model.predict(test_data)
print(metrics.classification_report(test_target, test_est2))  # 计算评估指标

[0.5]
[0.5]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        11

   micro avg       1.00      1.00      1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [67]:
best_model #C  gamma  kernel

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.5, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
best_model.score(train_data, train_target)

0.925

In [69]:
test_est2

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1])